<a href="https://colab.research.google.com/github/jjooki/TIL/blob/main/tensorflow_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model, layers, optimizers, datasets

In [ ]:
(images_train, labels_train), (images_test, labels_test) = datasets.mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
images_train = images_train/images_train.max()
images_test = images_test/images_test.max()

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.0 MB/s 
     |████████████████████████████████| 168 kB 49.7 MB/s 
     |████████████████████████████████| 182 kB 64.7 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 
     |████████████████████████████████| 168 kB 44.9 MB/s 
     |████████████████████████████████| 166 kB 17.7 MB/s 
     |████████████████████████████████| 166 kB 28.6 MB/s 
     |████████████████████████████████| 162 kB 43.6 MB/s 
     |████████████████████████████████| 162 kB 3.7 MB/s 
     |████████████████████████████████| 158 kB 34.1 MB/s 
     |████████████████████████████████| 157 kB 29.9 MB/s 
     |████████████████████████████████| 157 kB 50.1 MB/s 
     |████████████████████████████████| 157 kB 53.3 MB/s 
     |████████████████████████████████| 157 kB 45.5 MB/s 
     |████████████████████████████████| 157 kB 33.0 MB/s 
     |█████████████████████████████

In [ ]:
# import os
# print(os.environ['WANDB_API_KEY'])

KeyError: ignored

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
args = dict(
    total_epochs=3,
    batch_size=128,
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics='accuracy',
    learning_rate=0.005,
    num_classes=10,
    hidden_size=[128, 256]
)

In [ ]:
run = wandb.init(project='fastcampus_dl(Tensorflow)')
wandb.config.update(args)

wandb: Currently logged in as: piy8117982. Use `wandb login --relogin` to force relogin


In [ ]:
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(args['hidden_size'][0], activation='relu'),
    layers.Dense(args['hidden_size'][1], activation='relu'),
    layers.Dense(args['num_classes'], activation='softmax')
])

In [ ]:
model.compile(optimizer=args['optimizer'],
              loss=args['loss'],
              metrics=args['metrics'])

In [ ]:
model.fit(images_train, labels_train,
          validation_data=(images_test, labels_test),
          epochs=args['total_epochs'],
          batch_size=args['batch_size'],
          callbacks=[WandbCallback(data_type='image', labels=labels_train)])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING The data_type argument of wandb.keras.WandbCallback is deprecated and will be removed in a future release. Please use input_type instead.
wandb: WARNING Setting input_type = data_type.


Epoch 1/3
2/2 [==============================] - 0s 10ms/step


wandb: Adding directory to artifact (/content/wandb/run-20221129_184402-29x4kzd8/files/model-best)... Done. 0.1s


469/469 [==============================] - 12s 23ms/step - loss: 0.3009 - accuracy: 0.9144 - val_loss: 0.1527 - val_accuracy: 0.9535
Epoch 2/3
2/2 [==============================] - 0s 10ms/step


wandb: Adding directory to artifact (/content/wandb/run-20221129_184402-29x4kzd8/files/model-best)... Done. 0.1s


469/469 [==============================] - 8s 16ms/step - loss: 0.1197 - accuracy: 0.9643 - val_loss: 0.1127 - val_accuracy: 0.9642
Epoch 3/3
2/2 [==============================] - 0s 7ms/step


wandb: Adding directory to artifact (/content/wandb/run-20221129_184402-29x4kzd8/files/model-best)... Done. 0.1s


469/469 [==============================] - 8s 16ms/step - loss: 0.0816 - accuracy: 0.9756 - val_loss: 0.0852 - val_accuracy: 0.9728


In [ ]:
loss, accuracy = model.evaluate(images_test, labels_test)
wandb.log({'test loss': loss})
wandb.log({'test accuracy': accuracy})

313/313 [==============================] - 1s 4ms/step - loss: 0.0852 - accuracy: 0.9728
